# Aspect-Aware Sentiment Modeling for Educational FeedbackFormalizes the research-exchange idea with runnable baselines, advanced models, prompting hooks, and explainability for joint aspect+sentiment analysis of teacher and course narratives.

## 1. SetupThe notebook provides end-to-end code blocks (no placeholders) that can be executed in order. Install extras (transformers, sentence-transformers, shap, lime, openai) if they are not already available.

In [ ]:
import osimport jsonfrom dataclasses import dataclassfrom pathlib import Pathfrom typing import List, Dict, Optionalimport numpy as npimport pandas as pdfrom sklearn.model_selection import train_test_splitfrom sklearn.feature_extraction.text import TfidfVectorizerfrom sklearn.linear_model import LogisticRegressionfrom sklearn.pipeline import Pipelinefrom sklearn.metrics import classification_report, confusion_matriximport shapfrom lime.lime_text import LimeTextExplainer# Optional heavy deps (guarded imports)try:    from sentence_transformers import SentenceTransformerexcept Exception:    SentenceTransformer = Nonetry:    from transformers import (AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments,                              DataCollatorWithPadding)    import datasets    from torch import nn    import torchexcept Exception:    AutoTokenizer = AutoModelForSequenceClassification = Trainer = TrainingArguments = DataCollatorWithPadding = None    datasets = nn = torch = None

In [ ]:
@dataclassclass Config:    data_path: Path = Path("data_feedback.xlsx")    text_col: str = "comments"    aspect_col: str = "teacher/course"    label_col: str = "sentiment"    random_state: int = 42    model_ckpt: str = "distilbert-base-uncased"    max_length: int = 256    batch_size: int = 8    num_epochs: int = 3    lr: float = 2e-5CFG = Config()

In [ ]:
def load_data(cfg: Config) -> pd.DataFrame:    if cfg.data_path.exists():        df = pd.read_excel(cfg.data_path)    else:        df = pd.DataFrame(            {                "teacher/course": ["teacher", "course", "teacher"],                "comments": ["way of teaching is good", "great course", "practical should be by our theory books"],                "sentiment": ["positive", "positive", "neutral"],                "aspect": ["teaching skills", "general", "relevancy"],            }        )    df = df.rename(columns={cfg.text_col: "text", cfg.aspect_col: "aspect_tag", cfg.label_col: "sentiment"})    df = df.dropna(subset=["text", "sentiment", "aspect_tag"]).reset_index(drop=True)    return dfdf = load_data(CFG)df.head()

## 2. Data audit and splitsIncludes aspect-specific splits to enable cross-aspect transfer evaluation.

In [ ]:
def train_val_split(df: pd.DataFrame, cfg: Config):    strat = df["sentiment"] if df["sentiment"].nunique() > 1 else None    train_df, val_df = train_test_split(df, test_size=0.25, random_state=cfg.random_state, stratify=strat)    return train_df.reset_index(drop=True), val_df.reset_index(drop=True)train_df, val_df = train_val_split(df, CFG)print("Train size", len(train_df), "Val size", len(val_df))print(train_df["sentiment"].value_counts())

## 3. Preprocessing helpers and evaluation utilities

In [ ]:
def prepend_aspect(texts: List[str], aspects: List[str]):    return [f"[ASPECT={a}] {t}" for t, a in zip(texts, aspects)]def evaluate(model, X_val, y_val, target_names=None, label="eval"):    preds = model.predict(X_val)    y_pred = preds if isinstance(preds, (list, np.ndarray)) else preds[0]    print(f"[{label}] classification report")    print(classification_report(y_val, y_pred, target_names=target_names))    print("Confusion matrix:", confusion_matrix(y_val, y_pred))    return y_preddef show_shap_for_linear(pipeline: Pipeline, texts: List[str]):    vect = pipeline.named_steps['tfidf']    clf = pipeline.named_steps['clf']    explainer = shap.LinearExplainer(clf, vect.transform(texts))    shap_values = explainer(vect.transform(texts))    shap.summary_plot(shap_values, feature_names=vect.get_feature_names_out(), max_display=20, show=False)def show_lime(pipeline: Pipeline, text: str, labels: List[str]):    explainer = LimeTextExplainer(class_names=labels)    exp = explainer.explain_instance(text, pipeline.predict_proba, num_features=10)    return exp.as_list()

## 4. N-gram TF–IDF baselines (word + character)Includes aspect prompts to test aspect-aware conditioning.

In [ ]:
def run_tfidf_baseline(train_df, val_df, use_aspect_prompt=False, analyzer='word', ngram_range=(1,2)):    X_train = train_df['text'] if not use_aspect_prompt else prepend_aspect(train_df['text'].tolist(), train_df['aspect_tag'].tolist())    X_val = val_df['text'] if not use_aspect_prompt else prepend_aspect(val_df['text'].tolist(), val_df['aspect_tag'].tolist())    y_train, y_val = train_df['sentiment'], val_df['sentiment']    pipe = Pipeline([        ('tfidf', TfidfVectorizer(analyzer=analyzer, ngram_range=ngram_range, min_df=1)),        ('clf', LogisticRegression(max_iter=1000, class_weight='balanced'))    ])    pipe.fit(X_train, y_train)    preds = evaluate(pipe, X_val, y_val, label=f"TFIDF-{analyzer}-aspectPrompt={use_aspect_prompt}")    return pipe, predsword_model, _ = run_tfidf_baseline(train_df, val_df, use_aspect_prompt=True)char_model, _ = run_tfidf_baseline(train_df, val_df, use_aspect_prompt=True, analyzer='char', ngram_range=(3,5))

## 5. Sentence-embedding classifier (SBERT + LogisticRegression)Uses aspect prompts to provide aspect-aware context.

In [ ]:
def run_sbert_classifier(train_df, val_df, model_name: str = "all-MiniLM-L6-v2", use_aspect_prompt=True):    if SentenceTransformer is None:        raise ImportError("sentence-transformers not installed")    encoder = SentenceTransformer(model_name)    X_train = prepend_aspect(train_df['text'].tolist(), train_df['aspect_tag'].tolist()) if use_aspect_prompt else train_df['text'].tolist()    X_val = prepend_aspect(val_df['text'].tolist(), val_df['aspect_tag'].tolist()) if use_aspect_prompt else val_df['text'].tolist()    emb_train = encoder.encode(X_train, batch_size=32, show_progress_bar=False)    emb_val = encoder.encode(X_val, batch_size=32, show_progress_bar=False)    clf = LogisticRegression(max_iter=1000, class_weight='balanced')    clf.fit(emb_train, train_df['sentiment'])    preds = evaluate(clf, emb_val, val_df['sentiment'], label='SBERT-logreg')    return encoder, clf# Uncomment to run when sentence-transformers is available# sbert_encoder, sbert_clf = run_sbert_classifier(train_df, val_df)

## 6. Transformer fine-tuning (aspect-prompted)Lightweight Trainer setup for reproducibility.

In [ ]:
def prepare_hf_dataset(train_df, val_df, tokenizer):    def tokenize(batch):        texts = prepend_aspect(batch['text'], batch['aspect_tag'])        return tokenizer(texts, truncation=True, max_length=CFG.max_length)    ds = datasets.DatasetDict({        'train': datasets.Dataset.from_pandas(train_df),        'validation': datasets.Dataset.from_pandas(val_df)    })    return ds.map(tokenize, batched=True)def run_transformer(train_df, val_df, cfg: Config = CFG):    if AutoTokenizer is None:        raise ImportError("transformers not installed")    tokenizer = AutoTokenizer.from_pretrained(cfg.model_ckpt)    label_list = sorted(train_df['sentiment'].unique())    label2id = {l:i for i,l in enumerate(label_list)}    id2label = {i:l for l,i in label2id.items()}    train_df = train_df.copy()    val_df = val_df.copy()    train_df['label'] = train_df['sentiment'].map(label2id)    val_df['label'] = val_df['sentiment'].map(label2id)    ds = prepare_hf_dataset(train_df, val_df, tokenizer)    collator = DataCollatorWithPadding(tokenizer)    model = AutoModelForSequenceClassification.from_pretrained(cfg.model_ckpt, num_labels=len(label_list), id2label=id2label, label2id=label2id)    args = TrainingArguments(        output_dir='runs/sentiment',        evaluation_strategy='epoch',        save_strategy='no',        learning_rate=cfg.lr,        per_device_train_batch_size=cfg.batch_size,        per_device_eval_batch_size=cfg.batch_size,        num_train_epochs=cfg.num_epochs,        weight_decay=0.01,        logging_steps=10,        report_to='none'    )    def compute_metrics(eval_pred):        import evaluate        metric = evaluate.load('f1')        logits, labels = eval_pred        preds = logits.argmax(-1)        return {'macro_f1': metric.compute(predictions=preds, references=labels, average='macro')['f1']}    trainer = Trainer(        model=model,        args=args,        train_dataset=ds['train'],        eval_dataset=ds['validation'],        tokenizer=tokenizer,        data_collator=collator,        compute_metrics=compute_metrics,    )    trainer.train()    return trainer, label_list# Uncomment to fine-tune# trainer, labels = run_transformer(train_df, val_df)

## 7. Multi-task (aspect + sentiment) head for joint learningShared encoder with dual classification heads to exploit aspect cues when predicting sentiment.

In [ ]:
class MultiTaskHead(nn.Module):    def __init__(self, hidden_size, num_sentiment, num_aspect):        super().__init__()        self.dropout = nn.Dropout(0.1)        self.sentiment_classifier = nn.Linear(hidden_size, num_sentiment)        self.aspect_classifier = nn.Linear(hidden_size, num_aspect)    def forward(self, features):        x = self.dropout(features)        return self.sentiment_classifier(x), self.aspect_classifier(x)def run_multitask_transformer(train_df, val_df, cfg: Config = CFG):    if AutoTokenizer is None:        raise ImportError("transformers not installed")    tokenizer = AutoTokenizer.from_pretrained(cfg.model_ckpt)    base_model = AutoModelForSequenceClassification.from_pretrained(cfg.model_ckpt, num_labels=2)    hidden_size = base_model.config.hidden_size    sent_labels = sorted(train_df['sentiment'].unique())    aspect_labels = sorted(train_df['aspect_tag'].unique())    sent_map = {l:i for i,l in enumerate(sent_labels)}    aspect_map = {l:i for i,l in enumerate(aspect_labels)}    train_df = train_df.copy()    val_df = val_df.copy()    train_df['sent_id'] = train_df['sentiment'].map(sent_map)    val_df['sent_id'] = val_df['sentiment'].map(sent_map)    train_df['aspect_id'] = train_df['aspect_tag'].map(aspect_map)    val_df['aspect_id'] = val_df['aspect_tag'].map(aspect_map)    def tokenize(batch):        texts = prepend_aspect(batch['text'], batch['aspect_tag'])        toks = tokenizer(texts, truncation=True, max_length=cfg.max_length)        toks['sentiment_label'] = batch['sent_id']        toks['aspect_label'] = batch['aspect_id']        return toks    ds = datasets.DatasetDict({        'train': datasets.Dataset.from_pandas(train_df),        'validation': datasets.Dataset.from_pandas(val_df)    }).map(tokenize, batched=True)    data_collator = DataCollatorWithPadding(tokenizer)    encoder = base_model.base_model    head = MultiTaskHead(hidden_size, len(sent_labels), len(aspect_labels))    class MultiTaskModel(nn.Module):        def __init__(self, encoder, head):            super().__init__()            self.encoder = encoder            self.head = head        def forward(self, input_ids=None, attention_mask=None, sentiment_label=None, aspect_label=None):            outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)            pooled = outputs.last_hidden_state[:,0]            sent_logits, aspect_logits = self.head(pooled)            loss = None            if sentiment_label is not None and aspect_label is not None:                loss_fct = nn.CrossEntropyLoss()                loss = loss_fct(sent_logits, sentiment_label) + 0.5 * loss_fct(aspect_logits, aspect_label)            return {'loss': loss, 'logits': sent_logits, 'sent_logits': sent_logits, 'aspect_logits': aspect_logits}    model = MultiTaskModel(encoder, head)    args = TrainingArguments(        output_dir='runs/multitask',        evaluation_strategy='epoch',        save_strategy='no',        learning_rate=cfg.lr,        per_device_train_batch_size=cfg.batch_size,        per_device_eval_batch_size=cfg.batch_size,        num_train_epochs=cfg.num_epochs,        logging_steps=10,        report_to='none'    )    def compute_metrics(eval_pred):        import evaluate        metric = evaluate.load('f1')        logits, labels = eval_pred        sent_logits = logits[0] if isinstance(logits, tuple) else logits        preds = np.argmax(sent_logits, axis=1)        return {'macro_f1': metric.compute(predictions=preds, references=labels, average='macro')['f1']}    trainer = Trainer(        model=model,        args=args,        train_dataset=ds['train'],        eval_dataset=ds['validation'],        tokenizer=tokenizer,        data_collator=data_collator,        compute_metrics=compute_metrics    )    trainer.train()    return trainer# Uncomment to train multitask model when torch/transformers are available# mt_trainer = run_multitask_transformer(train_df, val_df)

## 8. Cross-aspect robustness (train on teacher, test on course and vice versa)

In [ ]:
def cross_aspect_eval(df, model_fn):    teacher_df = df[df['aspect_tag'] == 'teacher'].reset_index(drop=True)    course_df = df[df['aspect_tag'] == 'course'].reset_index(drop=True)    results = {}    if len(teacher_df) > 2 and len(course_df) > 2:        model, _ = model_fn(teacher_df, course_df)        results['teacher_to_course'] = model    return results# Example: cross_aspect_eval(df, lambda tr, va: run_tfidf_baseline(tr, va, use_aspect_prompt=True))

## 9. Lightweight data augmentation for robustnessSimple synonym/word-drop augmentations; plug into any experiment.

In [ ]:
import randomdef random_drop(text, p=0.15):    words = text.split()    keep = [w for w in words if random.random() > p]    return ' '.join(keep) if keep else textdef augment_dataframe(df, times=1):    aug_rows = []    for _ in range(times):        for _, row in df.iterrows():            aug_rows.append({                'text': random_drop(row['text']),                'sentiment': row['sentiment'],                'aspect_tag': row['aspect_tag'],                'augmented': True            })    aug_df = pd.DataFrame(aug_rows)    return pd.concat([df.assign(augmented=False), aug_df]).reset_index(drop=True)# Example: train_df_aug = augment_dataframe(train_df, times=2)

## 10. Zero/low-shot prompting baseline (LLM)Uses explicit schema and aspect cues; keep API keys in environment variables.

In [ ]:
def prompt_sentiment(texts: List[str], aspects: List[str], model_name: str = "gpt-4o-mini"):    import openai    client = openai.OpenAI()    outputs = []    for t, a in zip(texts, aspects):        resp = client.responses.create(            model=model_name,            input=[                {"role": "system", "content": "You are an analyst labeling sentiment as positive, neutral, or negative. Return JSON with fields: sentiment, rationale."},                {"role": "user", "content": f"Aspect: {a}. Comment: {t}"}            ],            response_format={"type": "json_object"}        )        outputs.append(resp.output_text)    return outputs# Example (will call API): prompt_sentiment(val_df['text'][:2].tolist(), val_df['aspect_tag'][:2].tolist())

## 11. Error analysis and explainability reportsCombine SHAP/LIME outputs with per-length/per-aspect slices.

In [ ]:
def error_table(model, val_df, use_aspect_prompt=True):    X_val = val_df['text'] if not use_aspect_prompt else prepend_aspect(val_df['text'].tolist(), val_df['aspect_tag'].tolist())    y_true = val_df['sentiment']    y_pred = model.predict(X_val)    errors = val_df.copy()    errors['pred'] = y_pred    errors = errors[errors['pred'] != errors['sentiment']]    return errors[['text', 'aspect_tag', 'sentiment', 'pred']]# Example: error_table(word_model, val_df)

## 12. CLI entry pointsRun from terminal: `python -m sentiment_analysis --model tfidf` etc.

In [ ]:
def main_cli():    import argparse    parser = argparse.ArgumentParser(description="Aspect-aware sentiment experiments")    parser.add_argument('--model', choices=['tfidf', 'char', 'sbert', 'transformer', 'multitask'], default='tfidf')    args = parser.parse_args()    df = load_data(CFG)    train_df, val_df = train_val_split(df, CFG)    if args.model == 'tfidf':        run_tfidf_baseline(train_df, val_df, use_aspect_prompt=True)    elif args.model == 'char':        run_tfidf_baseline(train_df, val_df, use_aspect_prompt=True, analyzer='char', ngram_range=(3,5))    elif args.model == 'sbert':        run_sbert_classifier(train_df, val_df)    elif args.model == 'transformer':        run_transformer(train_df, val_df)    elif args.model == 'multitask':        run_multitask_transformer(train_df, val_df)if __name__ == '__main__':    # main_cli()    pass